# Imports

In [1]:
import nltk
import numpy as np
import pandas as pd
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import gutenberg
import re
import spacy
import warnings
warnings.filterwarnings("ignore")

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'cornell_movie_dialogs'
conn_str = f'postgresql://{postgres_user}:{postgres_pw}@{postgres_host}:{postgres_port}/{postgres_db}'
query = '''
SELECT *
FROM dialogs
'''
dialogs = pd.read_sql_query(query, con=conn_str)
dialogs.head()

,index,dialogs
0,0,Can we make this quick? Roxanne Korrine and A...
1,1,"Well, I thought we'd start with pronunciation,..."
2,2,Not the hacking and gagging and spitting part....
3,3,Okay... then how 'bout we try out some French ...
4,4,You're asking me out. That's so cute. What's ...


1. First, do some data preprocessing to clean up the data. You can use your solution to the assignment of the Text preprocessing checkpoint.

2. Develop a chatbot using this corpus. In doing this, you're free to choose a chatbot development library like ChatterBot or write your own code from scratch.

3. Start a conversation with your chatbot, and discuss its strengths and weaknesses.

# Preprocessing

In [3]:
# function to create a doc from the dialogs data
sentences = dialogs.dialogs
def create_doc(sentences):
    return " ".join(sentences)

In [4]:
doc = create_doc(sentences)

nlp = spacy.load('en', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 20000000
doc = nlp(doc)

In [6]:
doc_sents = [sent.text for sent in doc.sents if len(sent.text) > 1]

In [10]:
!pip install chatterbot
!pip install chatterbot-corpus

     |████████████████████████████████| 71kB 3.3MB/s 
     |████████████████████████████████| 122kB 4.2MB/s 


In [11]:
# Import libraries
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement

In [12]:
# Create a chatbot
chatbot = ChatBot('Dennis')
# This is to remove the accumulated knowledge base
chatbot.storage.drop()

# Create a new trainer for the chatbot
trainer = ListTrainer(chatbot)

# Train the chatbot based on Persuasion
trainer.train(doc_sents)

List Trainer: [####################] 100%


In [13]:
print("Persuasion: I will try to respond to you reasonably. If you want to exit, type bye.")

# Below is the chatting
while True:
    
    user_input = input("User: ")
    user_input=user_input.lower()
    
    if(user_input!='bye'):
        if(user_input == 'thanks' or user_input == 'thank you'):
            break
            print("Persuasion: You're welcome.")
        else:
            if(greeting(user_input) != None):
                print("Persuasion: " + greeting(user_input))
            else:
                print("Persuasion: ", end = "")
                print(chatbot.get_response(user_input))
    else:
        print("Persuasion: Bye! It was a great chat.")
        break

Persuasion: I will try to respond to you reasonably. If you want to exit, type bye.
User: Hello
Persuasion: hey
User: How are you?
Persuasion: Takin' a bath.
User: Can bots get wet?
Persuasion: Tied and Tethered".
User: What does that mean?
Persuasion: Elevated!
User: Who is elevated?
Persuasion: You never wanted to go out with 'me, did you?
User: You never asked me to go out with you
Persuasion: Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.
User: Oh no, what happened?
Persuasion: Anything else?
User: I'm not sure, is there?
Persuasion: Where?
User: bye
Persuasion: Bye! It was a great chat.
